This part is for running AI model locally on photos from data folder 

In [1]:
import os
import json
import shutil
import requests
import firebase_admin
from firebase_admin import credentials, storage

In [ ]:
# === CONFIG ===
IMAGE_FOLDER = "COLLECTED_DATA"
OUTPUT_FOLDER = "OUTPUT_FOLDER"
ENDPOINT = "http://localhost:5001/model/detect_file"
NMS_VALUE = "0.2"
MIN_DOOR_SCORE = 0.35

WITH_DOOR_FOLDER = os.path.join(OUTPUT_FOLDER, "with_door")
NO_DOOR_FOLDER = os.path.join(OUTPUT_FOLDER, "no_door")
ANNOTATION_FOLDER = os.path.join(OUTPUT_FOLDER, "annotations")

# === Setup output folders ===
os.makedirs(WITH_DOOR_FOLDER, exist_ok=True)
os.makedirs(NO_DOOR_FOLDER, exist_ok=True)
os.makedirs(ANNOTATION_FOLDER, exist_ok=True)

# === Walk through all subfolders ===
for root, _, files in os.walk(IMAGE_FOLDER):
    image_files = [
        f for f in files if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    if not image_files:
        continue

    rel_dir = os.path.relpath(root, IMAGE_FOLDER)
    rel_output_subdir = os.path.normpath(rel_dir) if rel_dir != "." else ""

    num_processed = 0
    num_skipped = 0

    for filename in image_files:
        image_path = os.path.join(root, filename)
        annotation_subfolder = os.path.join(ANNOTATION_FOLDER, rel_output_subdir)
        os.makedirs(annotation_subfolder, exist_ok=True)

        annotation_path = os.path.join(
            annotation_subfolder, f"{os.path.splitext(filename)[0]}.json"
        )

        # === Skip if already processed
        if os.path.exists(annotation_path):
            print(f"⏭ Skipping {filename} (already processed)")
            num_skipped += 1
            continue

        print(f"🔍 Processing {image_path}...")

        # === Read and send image to endpoint
        try:
            with open(image_path, "rb") as f:
                files_payload = {"image": f}
                data = {"nms": NMS_VALUE}
                res = requests.post(ENDPOINT, files=files_payload, data=data)
                res.raise_for_status()
                detection_result = res.json()
        except Exception as e:
            print(f"❌ Error processing {filename}: {e}")
            continue

        # === Save annotation
        with open(annotation_path, "w") as out_f:
            json.dump(detection_result, out_f, indent=2)

        # === Check for 'door' object
        has_door = any(
            obj.get("type") == "door" and float(obj.get("score", 0)) >= MIN_DOOR_SCORE
            for obj in detection_result
        )

        if has_door:
            print(f"✅ {filename} has a DOOR with score >= {MIN_DOOR_SCORE}")
        else:
            print(f"🚫 {filename} has NO valid door detection")

        # === Move to correct folder
        dest_base = WITH_DOOR_FOLDER if has_door else NO_DOOR_FOLDER
        dest_folder = os.path.join(dest_base, rel_output_subdir)
        os.makedirs(dest_folder, exist_ok=True)

        shutil.copy(image_path, os.path.join(dest_folder, filename))
        print(f"📁 Copied to: {dest_folder}")
        num_processed += 1

    print(f"📂 Finished folder: {rel_dir} | ✅ Processed: {num_processed} | ⏭ Skipped: {num_skipped}")

print("🎉 All folders completed.")


⏭ Skipping 40.735609148509624,-73.99390356262795_TRACT_61_HEADING_298.22365063287407_ZOOM_58.jpg (already processed)
⏭ Skipping 40.73509931800212,-73.99427411682686_TRACT_61_HEADING_118.35866523037612_ZOOM_58.jpg (already processed)
⏭ Skipping 40.733540706426936,-73.99541309095922_TRACT_61_HEADING_298.57580924011904_ZOOM_58.jpg (already processed)
⏭ Skipping 40.73541147652532,-73.99404690044192_TRACT_61_HEADING_118.22365063287405_ZOOM_58.jpg (already processed)
⏭ Skipping 40.73393558555569,-73.99512259485167_TRACT_61_HEADING_298.57580924011904_ZOOM_58.jpg (already processed)
⏭ Skipping 40.73456627800977,-73.99466267018288_TRACT_61_HEADING_118.32201372386314_ZOOM_58.jpg (already processed)
⏭ Skipping 40.73590565612095,-73.99368855430025_TRACT_61_HEADING_298.22365063287407_ZOOM_58.jpg (already processed)
⏭ Skipping 40.73520597925978,-73.99419634305264_TRACT_61_HEADING_298.3586652303761_ZOOM_58.jpg (already processed)
⏭ Skipping 40.73477933388541,-73.9945074366444_TRACT_61_HEADING_118.358

KeyboardInterrupt: 

Connect to Firebase to upload images

Step 1. Connect to FireStorage

In [ ]:
# Initialize the app with your service account
cred = credentials.Certificate("")
firebase_admin.initialize_app(cred, {
    'storageBucket': ''
})

Step 2. Upload images from local folder to the storage

In [ ]:
import os
import uuid
from firebase_admin import storage

LOCAL_ROOT_FOLDER = 'OUTPUT_FOLDER/with_door'
IMAGE_EXTENSIONS = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp'}

bucket = storage.bucket()

def upload_folder_images_with_token(root_folder):
    for dirpath, _, filenames in os.walk(root_folder):
        for filename in filenames:
            ext = os.path.splitext(filename)[1].lower()
            if ext in IMAGE_EXTENSIONS:
                local_path = os.path.join(dirpath, filename)
                unique_name = os.path.basename(dirpath) + '_' + filename
                blob = bucket.blob(f'images/{unique_name}')

                # Upload file
                blob.upload_from_filename(local_path)
                blob.make_public()
                # Create a new token (UUID)
                token = str(uuid.uuid4())

                # Set metadata with download token
                metadata = blob.metadata or {}
                metadata['firebaseStorageDownloadTokens'] = token
                blob.metadata = metadata
                blob.patch()

                # Construct download URL
                download_url = (
                    f"https://firebasestorage.googleapis.com/v0/b/{bucket.name}/o/"
                    f"images%2F{unique_name.replace('/', '%2F')}?alt=media&token={token}"
                )


                print(f"✅ Uploaded {unique_name}")
                print(f"   Download URL: {download_url}")

if __name__ == "__main__":
    upload_folder_images_with_token(LOCAL_ROOT_FOLDER)


✅ Uploaded West28Street_40.74490299061111,-73.98782204746875_TRACT_58_HEADING_208.40353987143996_ZOOM_58_PITCH_-5.jpg
   Download URL: https://firebasestorage.googleapis.com/v0/b/door-front-test.firebasestorage.app/o/images%2FWest28Street_40.74490299061111,-73.98782204746875_TRACT_58_HEADING_208.40353987143996_ZOOM_58_PITCH_-5.jpg?alt=media&token=ccc69d3c-7473-41ca-b241-c6376f442281
✅ Uploaded West28Street_40.745878760474895,-73.99013232134486_TRACT_58_HEADING_28.373231271075376_ZOOM_58_PITCH_-5.jpg
   Download URL: https://firebasestorage.googleapis.com/v0/b/door-front-test.firebasestorage.app/o/images%2FWest28Street_40.745878760474895,-73.99013232134486_TRACT_58_HEADING_28.373231271075376_ZOOM_58_PITCH_-5.jpg?alt=media&token=0089542f-c6ab-4b20-bde7-bb29a496ac76
✅ Uploaded West28Street_40.745878760474895,-73.99013232134486_TRACT_58_HEADING_208.37323127107538_ZOOM_58_PITCH_-5.jpg
   Download URL: https://firebasestorage.googleapis.com/v0/b/door-front-test.firebasestorage.app/o/images%2

KeyboardInterrupt: 

Connect to MongoDB 

In [5]:
from pymongo import MongoClient

def get_mongo_collection():
    # Connect to local MongoDB server
    client = MongoClient("mongodb://localhost:27017/")

    # Access the database and collection
    db = client["myFirstDatabase_local"]
    #collection = db["auto-photo-collection"]
    collection = db["collect_panorama"]

    return collection


In [ ]:
import os
import re
from urllib.parse import quote
import pytz
from datetime import datetime
import uuid

# === PARSE METADATA ===
def parse_filename(filename):
    pattern = (
        r'(?P<street>.+?)_' +
        r'(?P<lat>-?\d+\.\d+),' +
        r'(?P<lon>-?\d+\.\d+)_TRACT_' +
        r'(?P<tract>\d+(\.\d+)?)_HEADING_' +
        r'(?P<heading>-?\d+(\.\d+)?)_ZOOM_' +
        r'(?P<zoom>\d+)' +
        r'(?:_PITCH_(?P<pitch>-?\d+(\.\d+)?))?' +   
        r'\.jpg'
    )

    match = re.match(pattern, filename)
    if match:
        pitch_str = match.group("pitch")
        pitch = float(pitch_str) if pitch_str is not None else -5

        return {
            "street_name": match.group("street"),
            "location": {
                "lat": float(match.group("lat")),
                "lng": float(match.group("lon")),
            },
            "tract": float(match.group("tract")),
            "pov": {
                "heading": float(match.group("heading")),
                "zoom": int(match.group("zoom")),
                "pitch": pitch
            }
        }
    return None

# === GET CURRENT TIME IN EASTERN TIMEZONE (NYC) ===
def get_eastern_time_now():
    eastern = pytz.timezone('America/New_York')
    return datetime.now(eastern)

# === LOAD MODEL LABELS ===

ANNOTATION_FOLDER="OUTPUT_FOLDER/annotations"
def load_model_labels_for_image(street_name, full_filename):
    """
    street_name: 'AvenueOfTheAmericas'
    full_filename: 'AvenueOfTheAmericas_40.737753,-73.9937_TRACT_54_HEADING_28.3_ZOOM_58.jpg'
    """
    # Remove '.jpg' and 'street_name_' prefix
    image_basename = full_filename.replace(".jpg", "").removeprefix(f"{street_name}_")

    annotation_path = os.path.join(ANNOTATION_FOLDER, street_name, image_basename + ".json")
    
    if not os.path.isfile(annotation_path):
        print(f"[!] Annotation not found: {annotation_path}")
        return []

    with open(annotation_path, 'r') as f:
        annotations = json.load(f)

    labels = []
    for ann in annotations:
        if "bbox" in ann and "type" in ann:
            x1, y1, x2, y2 = ann["bbox"]
            label_data = {
                "label_id": str(uuid.uuid4()),
                "box": {
                    "x": x1,
                    "y": y1,
                    "width": abs(x2 - x1),
                    "height": abs(y2 - y1)
                },
                "label": ann["type"],
                "labeledBy": "model"
            }
            labels.append(label_data)
    return labels

# === MAIN FUNCTION ===
def save_public_urls_to_mongo():
    mongo_collection = get_mongo_collection()

    for blob in bucket.list_blobs(prefix="images/"):
        filename = os.path.basename(blob.name)
        if not filename.endswith(".jpg"):
            continue

        metadata = parse_filename(filename)
        if metadata is None:
            print(f"[!] Skipping (unmatched): {filename}")
            continue

        FIREBASE_BUCKET = "door-front-test.firebasestorage.app"

        # Get download token from blob metadata
        blob.reload()  # make sure metadata is loaded
        token = None
        if blob.metadata and 'firebaseStorageDownloadTokens' in blob.metadata:
            token = blob.metadata['firebaseStorageDownloadTokens']

        if token:
            public_url = (
                f"https://firebasestorage.googleapis.com/v0/b/{FIREBASE_BUCKET}/o/"
                f"{quote(blob.name, safe='')}?alt=media&token={token}"
            )
        else:
            public_url = (
                f"https://firebasestorage.googleapis.com/v0/b/{FIREBASE_BUCKET}/o/"
                f"{quote(blob.name, safe='')}?alt=media"
            )

        current_time = get_eastern_time_now()

        document = {
            "image_id": filename,
            "firebase_path": blob.name,
            "url": public_url,
            "image_size": [640, 640],
            "creator": "auto-door",
            "createdAt": current_time,
            "updatedAt": current_time,
            **metadata
        }

    # (rest of your insertion code follows...)


        # Check if already exists
        if mongo_collection.find_one({"filename": filename}):
            print(f"[i] Already exists: {filename}")
            continue

        # Insert image metadata
        result = mongo_collection.insert_one(document)
        image_id = result.inserted_id

        # Derive JSON file name
        image_name = filename.replace(".jpg", "")
        street_name = metadata["street_name"]

        # Add model labels
        model_labels = load_model_labels_for_image(street_name, image_name)
        if model_labels:
            mongo_collection.update_one(
                {"_id": image_id},
                {"$set": {"model_labels": model_labels}}
            )
            print(f"[✓] Inserted image and labels: {filename}")
        else:
            print(f"[✓] Inserted image WITHOUT labels: {filename}")

if __name__ == "__main__":
    save_public_urls_to_mongo()


[✓] Inserted image and labels: AvenueOfTheAmericas_40.74172950077022,-73.99365770369627_TRACT_58_HEADING_118.26830982083908_ZOOM_58_PITCH_-5.jpg
[✓] Inserted image and labels: AvenueOfTheAmericas_40.7418302608825,-73.99358449693185_TRACT_58_HEADING_118.26830982083908_ZOOM_58_PITCH_-5.jpg
[✓] Inserted image and labels: AvenueOfTheAmericas_40.74193102094401,-73.99351128994428_TRACT_58_HEADING_118.26830982083908_ZOOM_58_PITCH_-5.jpg
[✓] Inserted image and labels: AvenueOfTheAmericas_40.74193102094401,-73.99351128994428_TRACT_58_HEADING_298.2683098208391_ZOOM_58_PITCH_-5.jpg
[✓] Inserted image and labels: AvenueOfTheAmericas_40.74203178095475,-73.99343808273359_TRACT_58_HEADING_118.26830982083908_ZOOM_58_PITCH_-5.jpg
[✓] Inserted image and labels: AvenueOfTheAmericas_40.74213254091469,-73.99336487529975_TRACT_58_HEADING_118.26830982083908_ZOOM_58_PITCH_-5.jpg
[✓] Inserted image and labels: AvenueOfTheAmericas_40.74213254091469,-73.99336487529975_TRACT_58_HEADING_298.2683098208391_ZOOM_58_P